# Import

In [51]:
import numpy as np
import pandas as pd
import datetime

import sys, os
sys.path.insert(0, os.path.abspath('scripts'))
import rus_table_parser

# Data parsing

In [52]:
DAY = datetime.datetime.now().strftime("%d").lstrip('0')
MONTH = datetime.datetime.now().strftime("%m").lstrip('0')
# DAY = 23
# MONTH = 5

# Save of "https://coronavirus-monitor.ru/coronavirus-v-rossii/" page
path = f'html_data/{DAY}-{MONTH}/Коронавирус в России. Онлайн карта распространения коронавируса в России..html'
parse_df = rus_table_parser.run_parsing(path)
parse_df

,Confirmed,Date,Deaths,Recovered,Region/City
0,163913.0,2020-05-24,1993.0,49840.0,Москва
1,33515.0,2020-05-24,329.0,5520.0,Московская область
2,13339.0,2020-05-24,133.0,3452.0,Санкт-Петербург
3,8047.0,2020-05-24,65.0,2398.0,Нижегородская область
4,4095.0,2020-05-24,84.0,3267.0,Республика Дагестан
...,...,...,...,...,...
80,155.0,2020-05-24,2.0,114.0,Севастополь
81,95.0,2020-05-24,0.0,47.0,Республика Алтай
82,68.0,2020-05-24,0.0,36.0,Сахалинская область
83,62.0,2020-05-24,0.0,10.0,Ненецкий автономный округ


# Preparing parsed data

In [53]:
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4241,2020-05-23,Челябинская область,Chelyabinsk region,74.0,67.0,0.0,65.0,2199.0,11.0,819.0
4242,2020-05-23,Чеченская Республика,Chechen Republic,95.0,20.0,1.0,16.0,1066.0,12.0,659.0
4243,2020-05-23,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,1.0,6.0,58.0,1.0,30.0
4244,2020-05-23,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,36.0,0.0,288.0,2025.0,5.0,686.0
4245,2020-05-23,Ярославская область,Yaroslavl region,76.0,115.0,0.0,49.0,2424.0,13.0,493.0


In [54]:
# Create day-columns
parse_df['Day-Confirmed'] = 0
parse_df['Day-Deaths'] = 0
parse_df['Day-Recovered'] = 0

In [55]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b')
parse_df['Region/City'] = parse_df['Region/City'].astype('str').str.strip('\u200b')

In [56]:
# Rename regions
rename_dict = {
    'Татарстан': 'Республика Татарстан',
    'Башкортостан': 'Республика Башкортостан',
    'Чувашская Республика': 'Республика Чувашия',
    'Камчатский край' : 'Камчатский край',
    'Ямало-Ненецкий автономный округ' : 'Ямало-Ненецкий АО',
    'Республика Северная Осетия — Алания' : 'Республика Северная Осетия - Алания',
    'Республика Алтай' : 'Республика Алтай'
}

def rename(row):
    name = row['Region/City'][0]
    return pd.Series(rename_dict[name] if name in rename_dict else name)

parse_df['Region/City'] = parse_df.reset_index().groupby('index').apply(rename).reset_index(drop=True)

In [57]:
# Minus a day from now if needed
parse_df['Date'] = pd.to_datetime(parse_df['Date'])
# parse_df['Date'] = parse_df['Date'] - pd.Timedelta(days=1)

In [58]:
# Fit day-columns
def upd(row):
    reg = row['Region/City']
    
    row['Day-Confirmed'] = row['Confirmed'] - rus_df[rus_df['Region/City'] == reg]['Confirmed'].max()
    row['Day-Deaths']    = row['Deaths'] - rus_df[rus_df['Region/City'] == reg]['Deaths'].max()
    row['Day-Recovered'] = row['Recovered'] - rus_df[rus_df['Region/City'] == reg]['Recovered'].max()
    
    row['Day-Confirmed'] = row['Confirmed'] if np.isnan(row['Day-Confirmed']) else row['Day-Confirmed']
    row['Day-Deaths']    = row['Deaths'] if np.isnan(row['Day-Deaths']) else row['Day-Deaths']
    row['Day-Recovered'] = row['Recovered'] if np.isnan(row['Day-Recovered']) else row['Day-Recovered']
    
    return row.drop('Region/City')

parse_df = parse_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [59]:
# Check for missed regions
parse_df[np.isnan(parse_df['Day-Confirmed'])]

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered


# Data checking

In [60]:
print('Russia Confirmed:', parse_df.groupby('Region/City')['Confirmed'].max().sum(),
      'Day-Confirmed:', parse_df.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', parse_df.groupby('Region/City')['Deaths'].max().sum(),
      'Day-Deaths:', parse_df.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', parse_df.groupby('Region/City')['Recovered'].max().sum(),
      'Day-Recovered:', parse_df.groupby('Region/City')['Day-Recovered'].sum().sum())

# rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
# rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
# rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']]
# # rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 344774.0 Day-Confirmed: 8627.0
Russia Deaths: 3547.0 Day-Deaths: 149.0
Russia Recovered: 113387.0 Day-Recovered: 5290.0


# Data saving

In [61]:
# Save full snapshot of current day
parse_df.to_csv(f'covid19-russia-cases-upd-full.csv', index=False)

In [62]:
# Filter regions without changes
parse_df = parse_df[(parse_df['Day-Confirmed'] != 0) |
                (parse_df['Day-Deaths'] != 0) |
                (parse_df['Day-Recovered'] != 0)]

In [63]:
# Save parsed data for future use
parse_df.to_csv(f'old_data/covid19-russia-cases-upd{DAY}-{MONTH}.csv', index=False)
parse_df.to_csv('covid19-russia-cases-upd.csv', index=False)